In [5]:
import numpy as np
import pandas as pd
import psycopg2 as pg
from questdb.ingress import Sender

In [2]:
with Sender('localhost', 9009) as sender:

    for id, temp, hum in zip(list(range(1, 10001)), list(np.random.normal(size=10000)), list(np.random.normal(size=10000))):
        sender.row(
            'sensors',
            symbols={'id': str(id)},
            columns={'temperature': temp , 'humidity':hum})
        sender.flush()

In [4]:
connection = pg.connect(user="admin",
                            password="quest",
                            host="127.0.0.1",
                            port="8812",
                            database="qdb",
                            options='-c statement_timeout=300000')

In [7]:
data = pd.read_sql("select * from sensors", connection)
data

,id,temperature,humidity,timestamp
0,1,0.124996,-0.083947,2022-10-10 14:22:11.511191
1,2,0.322517,0.571596,2022-10-10 14:22:11.513569
2,3,-0.554790,-0.423297,2022-10-10 14:22:11.513743
3,4,0.359332,-0.029046,2022-10-10 14:22:11.514020
4,5,-0.066914,0.188475,2022-10-10 14:22:11.518827
...,...,...,...,...
9995,9996,1.628377,0.031897,2022-10-10 14:22:11.930697
9996,9997,0.411982,-0.045776,2022-10-10 14:22:11.930715
9997,9998,-0.309799,-1.187557,2022-10-10 14:22:11.930733
9998,9999,-0.159428,0.331559,2022-10-10 14:22:11.930748


In [13]:
connection.cursor().execute('''delete from sensors where humidity < 10001  ''')

DatabaseError: unexpected token: from
LINE 1: delete from sensors where humidity < 10001  
               ^
